## Mystem

На прошлом семинаре мы начале говорить о морфологическом анализе с помощью Mystem и научились **лемматизировать** (приводить к начальной форме) слова с его помощью. Вот как это было:

In [1]:
# не забудьте сначала установить библиотеку pymystem3

from pymystem3 import Mystem

text = """На бледно-голубой эмали,
Какая мыслима в апреле,
Березы ветви поднимали
И незаметно вечерели"""

m = Mystem()
lemmas = m.lemmatize(text)

print(type(lemmas)) # функция возвращает список
print(''.join(lemmas))

<class 'list'>
на бледно-голубой эмаль,
какой мыслимый в апрель,
береза ветвь подымать
и незаметно вечерель



### POS-tagging

Вторая важнейшая составляющая морфологического анализа -- получение грамматической информации о слове, т.е. к какой части речи оно принадлежит и в какой форме употреблено в тексте. Например, в предыдущем предложении *морфологического* -- полное прилагательное в мр.р, ед.ч., Р.п. В компьютерной лингвистике это называется **частеречная разметка**, или **POS-tagging** (part of speech tagging).

В `pymystem3` за полный морфологический разбор, который включает в себя как лемму, так и грамматические теги, отвечает функция `analyze`. Она выдает список (list) слов, полученных на вход, внутри которого лежит словарь (dict) со следующими ключами:

* analysis -- собственно анализ
* text -- слово, полученное на вход

Значением ключа `analysis` является список, в который входят два словаря:

* gr -- грамматическая информация
* lex -- лемма

In [5]:
m.analyze('морфологического')

[{'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
    'lex': 'морфологический'}],
  'text': 'морфологического'},
 {'text': '\n'}]

Используя знания о структуре выдачи, попробуем достать только грамматику.

In [8]:
res = m.analyze('морфологического')
grammar = res[0]['analysis'][0]['gr']
print(grammar)

A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)


### Дизамбигуация

Можно заметить, что майстем предлагает целых три возможных разбора (не сомневаясь в части речи):

* вин,ед,полн,муж,од
* род,ед,полн,муж
* род,ед,полн,сред

Они омонимичны, т.е. пишутся и звучат одинаково, поэтому без контекста выбрать нужный разбор не сможет даже человек. Выбор одного варианта разбора при наличии омонимии форм называется **снятием омонимии**, или **дизамбигуацией**. Для того, чтобы снять омонимию, и человеку, и машине нужен контекст: очевидно, что если прилагательное стоит рядом с существительным мужского рода, то оно в форме мужского рода, а не в форме среднего. Несмотря на то, что в последней версии майстема есть функция дизамбигуации, она справляется только с омонимией частей речи, а вот внутри парадигмы омонимию разрешать не умеет.

In [39]:
# справятся и человек, и майстем

print(m.analyze("синтез нового белка")[4], m.analyze("рыжая белка")[2], sep='\n', end='\n')
print(m.analyze("рабочий сказал")[0], m.analyze("рабочие руки")[0], sep='\n')

{'analysis': [{'lex': 'белок', 'gr': 'S,муж,неод=род,ед'}], 'text': 'белка'}
{'analysis': [{'lex': 'белка', 'gr': 'S,жен,од=им,ед'}], 'text': 'белка'}
{'analysis': [{'lex': 'рабочий', 'gr': 'S,муж,од=им,ед'}], 'text': 'рабочий'}
{'analysis': [{'lex': 'рабочий', 'gr': 'A=(вин,мн,полн,неод|им,мн,полн)'}], 'text': 'рабочие'}


In [14]:
# человек справится, майстем -- нет

m.analyze('составляющие морфологического анализа')

[{'analysis': [{'gr': 'PART=', 'lex': 'нет'}], 'text': 'нет'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
    'lex': 'морфологический'}],
  'text': 'морфологического'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'анализ'}],
  'text': 'анализа'},
 {'text': '\n'}]

### Глокая куздра

Уже 90 лет без этого зверя, придуманного академиком Л.В. Щербой, не обходится ни один разговор о морфологии. Фраза *"Гло́кая ку́здра ште́ко будлану́ла бо́кра и курдя́чит бокрёнка"* иллюстрирует то, что мы можем проанализировать слово с грамматической точки зрения, ничего не зная о его смысле: например, очевидно, что "кудрячит" -- это глагол в 3 л. ед.ч. наст.вр. Для майстема (да и для машины в целом) любой текст, который мы ему подаем, -- это глокая куздра. Он определяет форму слова не по смыслу, а по формальным показателям в конце слова.

In [40]:
# вы бы проанализировали эту фразу так же, как майстем, или нет?
''.join(m.lemmatize("глокая куздра штеко будланула бокра и кудрячит бокренка")) 

'глокая куздра штеко будланул бокра и кудрячить бокренок\n'

In [43]:
m.analyze("глокая куздра штеко будланула бокра и кудрячит бокренка")

[{'analysis': [{'gr': 'S,ед,жен,неод=им', 'lex': 'глокая', 'qual': 'bastard'}],
  'text': 'глокая'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,ед,жен,неод=им', 'lex': 'куздра', 'qual': 'bastard'}],
  'text': 'куздра'},
 {'text': ' '},
 {'analysis': [{'gr': 'ADV=', 'lex': 'штеко', 'qual': 'bastard'}],
  'text': 'штеко'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,од=(вин,ед|род,ед)',
    'lex': 'будланул',
    'qual': 'bastard'}],
  'text': 'будланула'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,ед,жен,неод=им', 'lex': 'бокра', 'qual': 'bastard'}],
  'text': 'бокра'},
 {'text': ' '},
 {'analysis': [{'gr': 'CONJ=', 'lex': 'и'}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,несов,пе=непрош,ед,изъяв,3-л',
    'lex': 'кудрячить',
    'qual': 'bastard'}],
  'text': 'кудрячит'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=род,ед',
    'lex': 'бокренок',
    'qual': 'bastard'}],
  'text': 'бокренка'},
 {'text': '\n'}]

### Граммемы

**Граммемами** называются обозначения частей речи и других морфологических категорий в разборе. Последовательность граммем -- это и есть **грамматический тег слова**. Например, тег `V,несов,пе=непрош,ед,изъяв,3-л` состоит из следующих граммем:

* V
* несов
* пе
* непрош
* ед
* изъяв
* 3-л

Самое непритное в том, что существует много наборов граммем, или **тегсетов**, для обозначения одного и того же. У одного только майстема их [два](https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/): русский и английский. Своя система морфологической разметки и в [НКРЯ](http://www.ruscorpora.ru/corpora-morph.html). А уж если обратиться к другим языкам... К счастью, в рамках проекта *Universal Dependencies* предпринята попытка создания универсальной разметки для всех языков(вот [части речи](http://universaldependencies.org/u/pos/) и [все остальные граммемы](http://universaldependencies.org/u/feat/index.html), но увы, пока далеко не все ей пользуются. В общем, конвертирование тегов из одной системы в другую -- типичная головная боль компьютерного лингвиста. 

### Mystem vs НКРЯ

<img style="float: left;" src="mystem.png" width="400">

<img style="float: left;" src="ruscorpora.png" width="550">

### JSON

Размеченные тексты нужно как-то хранить. Для этого идеально подходит формат **JSON**, что расшифровывается как *JavaScript Object Notation*. Это текстовый формат, при этом его "предложения" легко преобразовать в питоновские (и не только питоновские) структуры данных, потому что они похожи на словари и массивы. JSON легко читается и человеком, и компьютером. Мы не будем углубляться в особенности этого формата (но если вам хочется, то загляните [вот в этот семинар](https://github.com/ancatmara/learnpython2017/blob/master/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%D1%8B/5.%20JSON.ipynb)), а лишь научимся записывать в json-файл готовый разбор и, наоборот, считывать разобранный текст из файла. 

#### Модуль json

В питоне есть стандартный модуль json. В основном из этого модуля используют такие функции:

* `loads` -- превратить строку в формате JSON в объект питона - словарь или массив. У этой функции один обязательный аргумент -- строка.
* `dumps` -- превратить питоновский словарь или массив в строку JSON. У этой функции один обязательный аргумент - словарь или массив.
* `load` -- прочитать файл и превратить JSON, который в нем находится, в объект питона. У этой функции два обязательных аргумента -- файл и объект питона.
* `dump` -- превратить питоновский словарь или массив в строку JSON и записать ее в файл. У этой функции два обязательных аргумента - файл и объект питона.

Теперь сделаем разбор и запишем его в файл.

In [4]:
import json

kuzdra = m.analyze("глокая куздра штеко будланула бокра и кудрячит бокренка")

with open('kuzdra.json', 'w', encoding='utf-8') as f:
    json.dump(kuzdra, f, ensure_ascii=False)

По указанному пути появился файл `kuzdra.json`. Его можно открыть в любом текстовом редакторе, например Notepad++, а можно загрузить его содержимое в программу

In [5]:
with open('kuzdra.json', 'r', encoding='utf-8') as f:
    kuzdra = json.loads(f.read())

In [58]:
type(kuzdra)

list

In [60]:
print(kuzdra)

[{'analysis': [{'lex': 'глокая', 'qual': 'bastard', 'gr': 'S,ед,жен,неод=им'}], 'text': 'глокая'}, {'text': ' '}, {'analysis': [{'lex': 'куздра', 'qual': 'bastard', 'gr': 'S,ед,жен,неод=им'}], 'text': 'куздра'}, {'text': ' '}, {'analysis': [{'lex': 'штеко', 'qual': 'bastard', 'gr': 'ADV='}], 'text': 'штеко'}, {'text': ' '}, {'analysis': [{'lex': 'будланул', 'qual': 'bastard', 'gr': 'S,муж,од=(вин,ед|род,ед)'}], 'text': 'будланула'}, {'text': ' '}, {'analysis': [{'lex': 'бокра', 'qual': 'bastard', 'gr': 'S,ед,жен,неод=им'}], 'text': 'бокра'}, {'text': ' '}, {'analysis': [{'lex': 'и', 'gr': 'CONJ='}], 'text': 'и'}, {'text': ' '}, {'analysis': [{'lex': 'кудрячить', 'qual': 'bastard', 'gr': 'V,несов,пе=непрош,ед,изъяв,3-л'}], 'text': 'кудрячит'}, {'text': ' '}, {'analysis': [{'lex': 'бокренок', 'qual': 'bastard', 'gr': 'S,муж,неод=род,ед'}], 'text': 'бокренка'}, {'text': '\n'}]


### Try - except

Это специальная условная конструкция, которая помогает обрабатывать исключения. При наличии обработки исключений программа не падает, когда возникает какая-нибудь ошибка, а продолжает работу. 

In [6]:
for word in kuzdra:
    print(word['analysis'][0]['gr'])

S,ед,жен,неод=им


KeyError: 'analysis'

Ошибка возникла из-за того, что в списке с разборами `kuzdra` помимо слов майстем выдает и пробелы, переносы строки и прочие служебные символы, а у них нет грамматического разбора и, следвательно, ключа `analysis` в словаре. Ошибка `KeyError` как раз значит, что ключа с указанным именем в словаре нет. Чтобы программа не падала, дойдя до пробела, нужно написать обработку исключения, т.е. указать, что программа должна сделать в случае возникновения такой ошибки. 

In [7]:
for word in kuzdra:
    try:
        print(word['analysis'][0]['gr'])
    except KeyError:  # тут название ошибки
        pass          # тут то, что нужно сделать в случае ее возникновения

S,ед,жен,неод=им
S,ед,жен,неод=им
ADV=
S,муж,од=(вин,ед|род,ед)
S,ед,жен,неод=им
CONJ=
V,несов,пе=непрош,ед,изъяв,3-л
S,муж,неод=род,ед


Иногда нужно прописать одно исключение сразу для нескольких ошибок. Тогда синтаксис будет следующим:

In [8]:
for word in kuzdra:
    try:
        print(word['analysis'][0]['gr'])
    except (KeyError, IndexError) as e:  # тут название ошибок
        pass                             # тут то, что нужно сделать в случае их возникновения

S,ед,жен,неод=им
S,ед,жен,неод=им
ADV=
S,муж,од=(вин,ед|род,ед)
S,ед,жен,неод=им
CONJ=
V,несов,пе=непрош,ед,изъяв,3-л
S,муж,неод=род,ед


### Задание

1. Взять какой-нибудь небольшой текст (например, рассказ или длинное стихотворение), сохранить его в txt-файл.
2. Загрузив текст из этого файла, сделать полный грамматический разбор и записать результаты в файл json.
3. Написать регулярное выражение, которое будет извлекать из тега только часть речи.
4. Пройтись циклом по списку с разборами, который выдал pymystem3, извлекая из каждого разбора форму слова и его часть речи и записывая их в новый словарь (форма -- ключ, часть речи -- значение).
5. Посчитать абсолютную частоту для всех частей речи, а зате относительнную частоту (абсолютная частота / длина текста).